In [1]:
#do the data things!
import numpy as np
import pandas as pd

#visualize
import matplotlib.pyplot as plt
import seaborn as sns

#my sql creds
from env import get_db_url
import os

# import text and create_engine from sqlalchemy
from sqlalchemy import text, create_engine

# for presentation purposes
import warnings
warnings.filterwarnings("ignore")

In [2]:
def check_file_exists(fn, query, url):
    """
    This function will:
    - check if file exists in my local directory, if not, pull from sql db
    - read the given `query`
    - return dataframe
    """
    if os.path.isfile(fn):
        print('csv file found and loaded')
        return pd.read_csv(fn, index_col=0)
    else: 
        print('creating df and exporting csv')
        df = pd.read_sql(query, url)
        df.to_csv(fn)
        return df

In [3]:
# it only has 900,223 rows and 6 columns
def get_aca_data():
    """
    This function will:
         get_aca reads in txt file to csv
    """
    df = pd.read_csv("anonymized-curriculum-access.txt", sep=' ')
    
    return df

In [4]:
def get_logs_data():
    """
    This function will:
        - from the connection made to the `curriculum_logs` DB
            - using the `get_db_url` from my wrangle module.
    """
    # How to import a database from MySQL
    url = get_db_url('curriculum_logs')

    query = """
    SELECT * 
    FROM curriculum_logs.logs as l 
    JOIN curriculum_logs.cohorts as c ON c.id = l.cohort_id;
    """

    filename = 'logs.csv'
    df = check_file_exists(filename, query, url)

    df = pd.read_sql(query, url)
    
    return df   

In [5]:
def get_logs1_data():
    """
    This function will:
        - from the connection made to the `curriculum_logs` DB
            - using the `get_db_url` from my wrangle module.
    """
    # How to import a database from MySQL
    url = get_db_url('curriculum_logs')

    query = """
    SELECT * 
    FROM curriculum_logs.logs;
    """

    filename = 'logs1.csv'
    df = check_file_exists(filename, query, url)

    df = pd.read_sql(query, url)
    
    return df 

In [6]:
# if you use MySQL we only get 847,330 rows and 15 columns. it use the log.csv
df = get_logs1_data()

creating df and exporting csv


In [7]:
# if you use MySQL we only get 847,330 rows and 15 columns. it use the log.csv
df = get_logs2_data()

NameError: name 'get_logs2_data' is not defined

In [ ]:
# if you use MySQL we only get 847,330 rows and 15 columns. it use the log.csv
df = get_logs_data()
df

In [ ]:
# if you use MySQL we only get 847,330 rows and 15 columns. it use the log.csv
df = get_logs_data()
df

In [ ]:
data_summary(df_mysql)

In [ ]:
# a function that show a summary of the dataset
def data_summary(df):
    # Print the shape of the DataFrame
    print(f'data shape: {df.shape}')
    # set all the columns names to a lowercase
    df.columns = df.columns.str.lower()
    # Create a summary DataFrame
    summary = pd.DataFrame(df.dtypes, columns=['data type'])
    # Calculate the number of missing values
    summary['#missing'] = df.isnull().sum().values 
    # Calculate the percentage of missing values
    summary['%missing'] = df.isnull().sum().values / len(df)* 100
    # Calculate the number of unique values
    summary['#unique'] = df.nunique().values
    # Create a descriptive DataFrame
    desc = pd.DataFrame(df.describe(include='all').transpose())
    # Add the minimum, maximum, and first three values to the summary DataFrame
    summary['count'] = desc['count'].values
    summary['mean'] = desc['mean'].values
    summary['std'] = desc['std'].values
    summary['min'] = desc['min'].values
    summary['25%'] = desc['25%'].values
    summary['50%'] = desc['50%'].values
    summary['75%'] = desc['75%'].values
    summary['max'] = desc['max'].values
    summary['first_value'] = df.loc[0].values
    summary['second_value'] = df.loc[1].values
    summary['third_value'] = df.loc[2].values
    
    # Return the summary DataFrame
    return summary

In [ ]:
data_summary(df)

In [ ]:
df.head()

In [ ]:
col_to_remove = ['id','slack','deleted_at']

In [ ]:
def remove_columns(df, col_to_remove):
    """
    This function will:
    - take in a df and list of columns (you need to create a list of columns that you would like to drop under the name 'cols_to_remove')
    - drop the listed columns
    - return the new df
    """
    df = df.drop(columns=col_to_remove)
    
    return df

In [ ]:
def handle_missing_values(df, prop_required_columns=0.5, prop_required_rows=0.75):
    """
    This function will:
    - take in: 
        - a dataframe
        - column threshold (defaulted to 0.5)
        - row threshold (defaulted to 0.75)
    - calculates the minimum number of non-missing values required for each column/row to be retained
    - drops columns/rows with a high proportion of missing values.
    - returns the new df
    """
    
    column_threshold = int(round(prop_required_columns * len(df.index), 0))
    df = df.dropna(axis=1, thresh=column_threshold)
    
    row_threshold = int(round(prop_required_rows * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=row_threshold)
    
    return df

In [ ]:
def data_prep(df, col_to_remove, prop_required_columns=0.5, prop_required_rows=0.75):
    """
    This function will:
    - take in: 
        - a dataframe
        - list of columns
        - column threshold (defaulted to 0.5)
        - row threshold (defaulted to 0.75)
    - removes unwanted columns
    - remove rows and columns that contain a high proportion of missing values
    - returns cleaned df
    """
    df = remove_columns(df, col_to_remove)
    df = handle_missing_values(df, prop_required_columns, prop_required_rows)
    
    # converts int to datetime
    df.date = pd.to_datetime(df.date)
    
    # rename the numbers for names
    df.program_id = df.program_id.replace({1: 'full_stack_java_php', 2: 'full_stack_java_java', 3: 'datascience', 4: 'front_end_web_dev'})
    return df

In [ ]:
df = data_prep(df, col_to_remove, prop_required_columns=0.5, prop_required_rows=0.75)

In [ ]:
# Don't add it to the function. It will remove more than the nulls
# drop any nulls in the dataset
df = df.dropna()

In [ ]:
df.info()

In [ ]:
df.head()

## 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

## count

In [ ]:
# Define a function to calculate count
def count(df, column):
    return df[column].value_counts()

In [ ]:
# ip count
count(df, 'ip').sort_values(ascending=False)

In [ ]:
# ip count
count(df, 'user_id').sort_values(ascending=False)

In [ ]:
def one_user_df_prep(df, user):
    df = df[df.user_id == user].copy()
    df.date = pd.to_datetime(df.date)
    df = df.set_index(df.date)
    df = df.sort_index()
    pages_one_user = df['path'].resample('d').count()
    return pages_one_user

In [ ]:
def compute_pct_b(pages_one_user, span, k, user):
    midband = pages_one_user.ewm(span=span).mean()
    stdev = pages_one_user.ewm(span=span).std()
    ub = midband + stdev*k
    lb = midband - stdev*k
    
    my_df = pd.concat([pages_one_user, midband, ub, lb], axis=1)
    my_df.columns = ['pages_one_user', 'midband', 'ub', 'lb']
    
    my_df['pct_b'] = (my_df['pages_one_user'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])
    my_df['user_id'] = user
    return my_df

In [ ]:
def plot_bands(my_df, user):
    fig, ax = plt.subplots(figsize=(12,8))
    ax.plot(my_df.index, my_df.pages_one_user, label='Number of Pages, User: '+str(user))
    ax.plot(my_df.index, my_df.midband, label = 'EMA/midband')
    ax.plot(my_df.index, my_df.ub, label = 'Upper Band')
    ax.plot(my_df.index, my_df.lb, label = 'Lower Band')
    ax.legend(loc='best')
    ax.set_ylabel('Number of Pages')
    plt.show()

In [ ]:
def find_anomalies(df, user, span, weight, plot=False):
    pages_one_user = one_user_df_prep(df, user)
    
    my_df = compute_pct_b(pages_one_user, span, weight, user)
    
    if plot:
        plot_bands(my_df, user)
    
    return my_df[my_df.pct_b>1]

In [ ]:
df[df.ip == '97.105.19.58']

In [ ]:
user=570
span=720
k=6

In [ ]:
find_anomalies(df, user, span, k)

In [ ]:
def find_all_anomalies(df, span, k):
    """
    Finds anomalies for all users in the provided DataFrame over a specified span 
    of time. An anomaly is defined as a value that is above the upper band, which 
    is calculated using the Exponential Moving Average (EMA or midband) and a 
    specified number of standard deviations.

    Parameters:
        df (pd.DataFrame): The original DataFrame, which should include a 'user_id' 
                           and a 'date' column.
        span (int): The span of the window for the EMA calculation, representing 
                    the number of time periods (e.g., 7 for a week, 30 for a month).
        k (int): The number of standard deviations to use when calculating the 
                 upper and lower bounds.

    Returns:
        anomalies (pd.DataFrame): A DataFrame containing the anomalies for all users. 
                                   Each row includes the original page visit data, 
                                   the EMA (midband), the upper and lower bounds (ub and lb), 
                                   the %b value (pct_b), and the user ID. Only rows where 
                                   pct_b > 1 (indicating an anomaly) are included. If no 
                                   anomalies are found for a user, no rows for that user 
                                   will be included in the DataFrame.
    """
    anomalies = pd.DataFrame()

    for u in df.user_id.unique():
        one_user = find_anomalies(df, u, span, k)
        anomalies = pd.concat([anomalies, one_user])

    return anomalies

In [ ]:
df_find_all = find_all_anomalies(df, span, k)
df_find_all
df_find_all.sort_values(by='pages_one_user', ascending=False)

In [ ]:
df[df.ip == '97.105.19.58']

In [ ]:
df[df.user_id == 11].date.max()

In [ ]:
df_11 = df[(df.user_id == 11) & (df.date == '2020-07-22')]
df_11

In [ ]:
plt.figure(figsize=(60,8))
df_11.time.hist()
plt.xticks(rotation =45)
plt.show()

In [ ]:
df[(df.user_id == 11) & (df.date == '2020-07-22')].start_date

## Frequency

In [ ]:
# Define a function to calculate frequency
def frequency(df, column):
    return df[column].value_counts(normalize=True)*100

In [ ]:
frequency(df, 'user_id').sort_values(ascending =False)

In [ ]:
frequency(df, 'ip').sort_values(ascending =False)

## Visualize

In [ ]:
# Define a function to visualize count
def visualize_count(df, column):
    df[column].value_counts().sort_values().plot(kind='barh')

In [ ]:
visualize_count(df, 'ip')

In [ ]:
# Calculate the count and percent for each IP
ip_count_df = df['ip'].value_counts().reset_index().rename(columns={'index': 'ip', 'ip': 'count'})
ip_count_df['percent'] = (ip_count_df['count'] / df.shape[0]) * 100
ip_count_df.head()

In [ ]:
# Define a function to visualize count
def visualize_percent(df, column):
    ip_count_df.set_index(column).percent.sort_values().plot.barh()

    plt.title('percent of each ip address')
    plt.show()

In [ ]:
visualize_percent(df, 'ip')